In [30]:
import onnx

# 加载原始的 ONNX 模型
model_path = r"runs\classify\train13\weights\best.onnx"
model = onnx.load(model_path)

# 修改输入格式为 1*640*640*3
model.graph.input[0].type.tensor_type.shape.dim[1].dim_value = 384
model.graph.input[0].type.tensor_type.shape.dim[2].dim_value = 640
model.graph.input[0].type.tensor_type.shape.dim[3].dim_value = 3
model.graph.node[0].input[0]="transpose_output_1"

In [31]:
transpose_node_1 = onnx.helper.make_node(
    "Transpose",
    inputs=["images"],
    outputs=["transpose_output_1"],
    perm=[0, 3, 1, 2]  # 调整维度顺序
)
model.graph.node.insert(0,transpose_node_1)

In [32]:
node = [x for x in model.graph.node if "output0" in x.output]
node[0].input.pop(0)
node[0].input.reverse()

In [33]:
model.graph.node[-1]

input: "/model.22/Reshape_7_output_0"
input: "/model.22/Sigmoid_output_0"
output: "output0"
name: "/model.22/Concat_6"
op_type: "Concat"
attribute {
  name: "axis"
  i: 1
  type: INT
}

In [25]:
model.graph.node[-1].output[0]="transpose_node_2"

In [26]:
transpose_node_2 = onnx.helper.make_node(
    "Transpose",
    inputs=["transpose_node_2"],
    outputs=["output"],
    perm=[0, 2, 1]  # 调整维度顺序
)
model.graph.node.append(transpose_node_2)

In [27]:
model.graph.output[0].type.tensor_type.shape.dim[0].dim_value=1
model.graph.output[0].type.tensor_type.shape.dim[1].dim_value=15120
model.graph.output[0].type.tensor_type.shape.dim[2].dim_value=19

In [28]:
model.graph.output[0].name="output"

In [29]:
onnx.save(model, model_path.replace(".onnx","_new.onnx"))

In [46]:
# model.graph.node[0].name
for (i,node) in enumerate(model.graph.node):
    if "Concat_341" in node.name:
        print(i)

342
